In [1]:
import pandas as pd
from collections import Counter
import datetime
import numpy as np
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [2]:
from sklearn import preprocessing
from sklearn.metrics import log_loss

In [3]:
df = pd.read_csv('../Output/a/train_step3_301_final.csv') # 数据加载
test_df = pd.read_csv('../Output/a/test_step3_301_final.csv') # A测试集数据

In [4]:
df.shape

(477284, 77)

将数据集按照context_timestamp的日期划分成训练集和测试集

18-23日的当成训练集，24日的当成测试集

In [5]:
df1 = df.drop(['context_time_day','context_timestamp','instance_id'], 1)
test_data = test_df.drop(['context_time_day','context_timestamp','is_trade'], 1)

In [6]:
df_label = df1.pop('is_trade')
df_data = df1
del df1

In [7]:
instance_df = test_data.pop('instance_id') # pd.series

rows = df_data.shape[0]
merge_data = df_data.append(test_data)

In [ ]:
merge_data.columns

In [8]:
zuhe_col = ['shop_review_star', 'shop_delivery_service', 'item_collected_sales',
            'item_collected_pv', 'item_sales_pv', 'user_gender_age', 'user_gender_occ', 'user_gender_star']
trade_rate_col = ['item_id_traderate','item_brand_id_traderate',
                  'shop_id_traderate', 'user_id_traderate','item_city_id_traderate']
cnt_col = ['item_id_cnt1', 'user_id_cnt1', 'shop_id_cnt1',
           'item_id_cnt', 'user_id_cnt', 'shop_id_cnt']
col_drop =   trade_rate_col

col_select = [c for c in merge_data.columns if c not in col_drop]

scale_data = preprocessing.scale(merge_data[col_select])
data = scale_data[0:rows]  # 训练集数据
test_data = scale_data[rows:] # 测试集数据

In [9]:
train_df = data[df['context_time_day']<=23]
test_df = data[df['context_time_day']>23]

In [10]:
train_df_label = df_label[df['context_time_day']<=23]
test_df_label = df_label[df['context_time_day']>23]

In [11]:
def generateResult(instance_df, test_prelabel, filename='../Output/LRpre_result_0321.txt'):
    result = pd.DataFrame({'instance_id':instance_df})
    result['predicted_score'] = test_prelabel
    result.to_csv(filename, index=False, sep=' ', line_terminator='\r')

# lightGBM

In [12]:
import lightgbm as lgb

In [ ]:
# 特征选择
# from sklearn.feature_selection import SelectFromModel
# embed_model = SelectFromModel(LGBM, prefit=True)
# train_df = embed_model.transform(train_df)
# test_df = embed_model.transform(test_df)

In [ ]:
from sklearn.grid_search import GridSearchCV

param_val1 = {'n_estimators': list(range(20,201,10))}
param_val2 = {'max_depth':list(range(2,15,1))}
param_val3 = { 'colsample_bytree':[0.7,0.72,0.74,0.76,0.78,0.8,0.82,0.84,0.86,0.88,0.90]}
param_val4 = {'subsample':[0.5,0.6,0.7,0.8,0.9,1]}
param_val = {'learning_rate':[0.005,0.01,0.02,0.05,0.1,0.2,0.5]}

In [13]:
LGBM = lgb.LGBMRegressor(max_depth=7, n_estimators=60, learning_rate=0.1, num_leaves=63)
# LGBM = lgb.LGBMClassifier(  max_depth=3,
#                             n_estimators = 120,
#                             learning_rate =0.05,     
#                             objective = 'binary',
#                             subsample=0.7,
#                             colsample_bytree=0.74,
#                             num_leaves=8
# )

In [ ]:
cv1 = GridSearchCV(estimator = LGBM,
                  param_grid = param_val2, scoring='roc_auc', iid=False)
cv1.fit(train_df, train_df_label)
cv1.grid_scores_, cv1.best_params_, cv1.best_score_

In [14]:
LGBM.fit(train_df, train_df_label)

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
       learning_rate=0.1, max_depth=7, min_child_samples=20,
       min_child_weight=0.001, min_split_gain=0.0, n_estimators=60,
       n_jobs=-1, num_leaves=63, objective=None, random_state=None,
       reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
       subsample_for_bin=200000, subsample_freq=1)

In [15]:
test_df_preLabel = LGBM.predict(test_df)
logLoss = log_loss(test_df_label, test_df_preLabel)
print (logLoss)

0.0823693404566


In [16]:
test_prelabel = LGBM.predict(test_data)

In [ ]:
test_df_preLabel = LGBM.predict_proba(test_df)[:,1]
logLoss = log_loss(test_df_label, test_df_preLabel)
print (logLoss)
# A 线下0.08332 线上0.08337
# A 线下0.0822370223983 线上0.08331

# B 线下0.0854069321492 线上

In [ ]:
test_prelabel = LGBM.predict_proba(test_data)[:,1]

In [17]:
generateResult(instance_df, test_prelabel, filename='../Output/a/LGBMpre_result.txt')

# GBDT
Gradient Boosting Decision Tree梯度提升决策树

In [ ]:
from sklearn import ensemble

In [ ]:
# n_estimators树的颗数; learning_rate学习率; max_depth树的深度
GBDT = ensemble.GradientBoostingRegressor(n_estimators=30, learning_rate=0.1, max_depth=7)

In [ ]:
GBDT.fit(train_df, train_df_label)

In [ ]:
test_df_preLabel = GBDT.predict(test_df)
logLoss = log_loss(test_df_label, test_df_preLabel)
print (logLoss)
# 线下0.0823620123728 线上 0.08362

In [ ]:
test_prelabel = GBDT.predict(test_data)

In [ ]:
generateResult(instance_df, test_prelabel, filename='../Output/GBDTpre_result.txt')

# XGBoost

In [ ]:
import xgboost as xgb

In [ ]:
XGB_model = xgb.XGBRegressor(n_estimators=60, learning_rate=0.1, max_depth=7)

In [ ]:
XGB_model.fit(train_df, train_df_label, eval_set=[(test_df, test_df_label)])

In [ ]:
test_df_preLabel = XGB_model.predict(test_df)
logLoss = log_loss(test_df_label, test_df_preLabel)
print (logLoss)
# 线下 0.0841361944155  线上 0.0830
# 线下 0.0825883856602 线上 0.08324

In [ ]:
test_prelabel = XGB_model.predict(test_data)

In [ ]:
generateResult(instance_df, test_prelabel, filename='../Output/XGBpre_result_0325.txt')

# 随机森林RF

Random Forest回归模型


In [ ]:
from sklearn import ensemble

In [ ]:
# n_estimators树的颗数; max_features寻找最佳划分的特征数; max_depth树的深度
RF = ensemble.RandomForestRegressor(n_estimators=30, max_features='log2', max_depth=7)

In [ ]:
RF.fit(train_df, train_df_label)

In [ ]:
test_df_preLabel = RF.predict(test_df)
logLoss = log_loss(test_df_label, test_df_preLabel)
print (logLoss)
# 线下 0.0828739759419 线下 0.08359

In [ ]:
test_prelabel = RF.predict(test_data)

In [ ]:
generateResult(instance_df, test_prelabel, filename='../Output/RFpre_result_0321.txt')